# 使用Embedding进行RAG回答
这个Notebook主要用于使用Embedding进行文章搜索，并应用文心大模型进行回答。

主要步骤有:
1. 获取要使用的后端的鉴权参数，并设置ACCESS_TOKEN，请参考[认证鉴权文档](../../docs/authentication.md)。
2. 读取准备好的文章（本地知识库文章），并且完成文本切片。
3. 调用文心百中语义模型获得知识库文章的Embedding。
4. 对生成好的知识库文章向量进行存储(Option)。
5. 根据问题对知识库文章进行检索。
6. 根据检索出来的段落进行回答。

## 1. 认证鉴权
获取要使用的后端的鉴权参数，并设置ACCESS_TOKEN，请参考[认证鉴权文档](../../docs/authentication.md)。

In [45]:
import time
import math
import erniebot
import faiss

from tqdm import tqdm
from typing import List

erniebot.api_type = 'aistudio'
erniebot.access_token = '<eb-access-token>'

def get_embedding(word: List[str]) -> List[float]:
    """
    获取单词的embedding向量

    Args:
        word (List[str]): 需要获取embedding向量的单词列表

    Returns:
        List[float]: 单词的embedding向量列表
    """
    if len(word) <= 16:
        embedding = erniebot.Embedding.create(
                                            model = 'ernie-text-embedding',
                                            input = word
                                            ).get_result()
    else:
        size = len(word)
        embedding = []
        for i in tqdm(range(math.ceil(size / 16))):
            embedding.extend(erniebot.Embedding.create(model = 'ernie-text-embedding', input = word[i*16:(i+1)*16]).get_result())
            time.sleep(1)
    return embedding

## 2. 文本切片(Chunk)
您可以选择使用默认的按字数切分文章或者自定义函数对文本进行切分，如自定义切片函数输入需为知识库文章的集合List[str]，输出为切片好的文章段落List[str]。

In [46]:
# 我们准备了一篇文章，来源为中华人民共和国立法法。
with open('./data/laws_rag.txt','r') as f:
    content = f.read()

def split_by_len(texts:List[str], split_token:int =384) -> List[str]:
    split_token = 384 # 文心支持最大长度
    idx = 0
    chunk = []

    for text in texts:
        while idx + split_token < len(text):
            temp_text = text[idx:idx + split_token]
            next_idx = temp_text.rfind('。') + 1
            if next_idx != 0: # 如果该切片中没有句号，则直接添加整个句子
                chunk.append(temp_text[:next_idx])
                idx = idx + next_idx
            else:
                chunk.append(temp_text)
                idx = idx + split_token

        chunk.append(text[idx:])
    
    return chunk

doc_chunk = split_by_len([content])


## 3. 获取知识库文本向量(Embedding)
调用文心百中语义模型获得Embedding并进行标准化。

In [54]:
doc_embedding = get_embedding(doc_chunk)
assert len(doc_embedding) == len(doc_chunk),"shape mismatch"

import numpy as np

def l2_normalization(embedding:np.ndarray) -> np.ndarray:
    if embedding.ndim == 1:
        return embedding / np.linalg.norm(embedding).reshape(-1,1)
    else:
        return embedding/np.linalg.norm(embedding,axis=1).reshape(-1,1)

doc_embedding = l2_normalization(np.array(doc_embedding))

100%|██████████| 4/4 [00:06<00:00,  1.68s/it]


## 4. 向量存储(Vector DB)

您可以选择直接存储为npy文件在本地，或者使用faiss存储（需先下载faiss `pip install faiss-cpu`）以便于之后重复调用。

In [55]:
np.save('./data/knowledge_embedding.npy',doc_embedding)

In [56]:
index_ip = faiss.IndexFlatIP(len(doc_embedding[0]))
index_ip.add(doc_embedding)
faiss.write_index(index_ip, './data/knowledge_embedding.index')

In [57]:
# 切片文本也需同步存储
with open('./data/knowledge.txt','w') as f:
    for chunk in doc_chunk:
        f.write(repr(chunk) + '\n')

## 5. 检索(Search)
文本相似度默认使用Cosine Similarity进行匹配，然后找到top_k个与问题最匹配的段落。

In [ ]:
query = '国务院如何制定行政法规'

# 将存储的文本读出
doc_chunk = []
with open('./data/knowledge.txt','r') as f:
    for line in f:
        doc_chunk.append(eval(line))


### 5.1 直接使用npy读取
如果文本向量直接存储为npy文件，则直接读取之后进行搜索

In [61]:
doc_embedding = np.load('./data/knowledge_embedding.npy')

def find_related_doc(query:str, origin_chunk:List[str], doc_embedding, top_k:int=5) -> List[int]:
    rank_scores = l2_normalization(np.array(get_embedding([query]))) @ doc_embedding.T
    top_k_similar = rank_scores.argsort()[0][-top_k:].tolist()[::-1]
    res = ''
    for i in range(top_k):
       
        print(f"Top {i+1}:{origin_chunk[top_k_similar[i]]}")
        print('-'*50)
        res += f"参考文章 {i+1}:{origin_chunk[top_k_similar[i]]}" + '\n\n'
    return res

related_doc = find_related_doc(query,doc_chunk,doc_embedding)

Top 1:

第七十一条 全国人民代表大会常务委员会工作机构加强立法宣传工作，通过多种形式发布立法信息、介绍情况、回应关切。

第三章 行政法规

第七十二条 国务院根据宪法和法律，制定行政法规。

行政法规可以就下列事项作出规定：

（一）为执行法律的规定需要制定行政法规的事项；

（二）宪法第八十九条规定的国务院行政管理职权的事项。

应当由全国人民代表大会及其常务委员会制定法律的事项，国务院根据全国人民代表大会及其常务委员会的授权决定先制定的行政法规，经过实践检验，制定法律的条件成熟时，国务院应当及时提请全国人民代表大会及其常务委员会制定法律。

第七十三条 国务院法制机构应当根据国家总体工作部署拟订国务院年度立法计划，报国务院审批。国务院年度立法计划中的法律项目应当与全国人民代表大会常务委员会的立法规划和立法计划相衔接。
--------------------------------------------------
Top 2:

应当制定地方性法规但条件尚不成熟的，因行政管理迫切需要，可以先制定地方政府规章。规章实施满两年需要继续实施规章所规定的行政措施的，应当提请本级人民代表大会或者其常务委员会制定地方性法规。

没有法律、行政法规、地方性法规的依据，地方政府规章不得设定减损公民、法人和其他组织权利或者增加其义务的规范。

第九十四条 国务院部门规章和地方政府规章的制定程序，参照本法第三章的规定，由国务院规定。

第九十五条 部门规章应当经部务会议或者委员会会议决定。

地方政府规章应当经政府常务会议或者全体会议决定。

第九十六条 部门规章由部门首长签署命令予以公布。

地方政府规章由省长、自治区主席、市长或者自治州州长签署命令予以公布。

第九十七条 部门规章签署公布后，及时在国务院公报或者部门公报和中国政府法制信息网以及在全国范围内发行的报纸上刊载。
--------------------------------------------------
Top 3:

第七十六条 行政法规的决定程序依照中华人民共和国国务院组织法的有关规定办理。

第七十七条 行政法规由总理签署国务院令公布。

有关国防建设的行政法规，可以由国务院总理、中央军事委员会主席共同签署国务院、中央军事委员会令公布。

第七十八条 行政法规签署公布后，及时在国务院公报

### 5.2 使用faiss读取
如果使用faiss存储，则使用以下方法读取向量库并进行匹配。

In [68]:
import faiss

index_ip = faiss.read_index('./data/knowledge_embedding.index') 

def find_related_doc(
        query:str, 
        origin_chunk:List[str], 
        index_ip:faiss.swigfaiss.IndexFlatIP, 
        top_k:int=5
    ) -> List[int]:
    
    D, I = index_ip.search(np.array(get_embedding([query])),top_k)
    top_k_similar = I.tolist()[0]
    res = ''
    for i in range(top_k):
       
        print(f"Top {i+1}:{origin_chunk[top_k_similar[i]]}")
        print('-'*50)
        res += f"参考文章 {i+1}:{origin_chunk[top_k_similar[i]]}" + '\n\n'
    return res

related_doc = find_related_doc(query,doc_chunk,index_ip)

Top 1:

第七十一条 全国人民代表大会常务委员会工作机构加强立法宣传工作，通过多种形式发布立法信息、介绍情况、回应关切。

第三章 行政法规

第七十二条 国务院根据宪法和法律，制定行政法规。

行政法规可以就下列事项作出规定：

（一）为执行法律的规定需要制定行政法规的事项；

（二）宪法第八十九条规定的国务院行政管理职权的事项。

应当由全国人民代表大会及其常务委员会制定法律的事项，国务院根据全国人民代表大会及其常务委员会的授权决定先制定的行政法规，经过实践检验，制定法律的条件成熟时，国务院应当及时提请全国人民代表大会及其常务委员会制定法律。

第七十三条 国务院法制机构应当根据国家总体工作部署拟订国务院年度立法计划，报国务院审批。国务院年度立法计划中的法律项目应当与全国人民代表大会常务委员会的立法规划和立法计划相衔接。
--------------------------------------------------
Top 2:

应当制定地方性法规但条件尚不成熟的，因行政管理迫切需要，可以先制定地方政府规章。规章实施满两年需要继续实施规章所规定的行政措施的，应当提请本级人民代表大会或者其常务委员会制定地方性法规。

没有法律、行政法规、地方性法规的依据，地方政府规章不得设定减损公民、法人和其他组织权利或者增加其义务的规范。

第九十四条 国务院部门规章和地方政府规章的制定程序，参照本法第三章的规定，由国务院规定。

第九十五条 部门规章应当经部务会议或者委员会会议决定。

地方政府规章应当经政府常务会议或者全体会议决定。

第九十六条 部门规章由部门首长签署命令予以公布。

地方政府规章由省长、自治区主席、市长或者自治州州长签署命令予以公布。

第九十七条 部门规章签署公布后，及时在国务院公报或者部门公报和中国政府法制信息网以及在全国范围内发行的报纸上刊载。
--------------------------------------------------
Top 3:

第七十六条 行政法规的决定程序依照中华人民共和国国务院组织法的有关规定办理。

第七十七条 行政法规由总理签署国务院令公布。

有关国防建设的行政法规，可以由国务院总理、中央军事委员会主席共同签署国务院、中央军事委员会令公布。

第七十八条 行政法规签署公布后，及时在国务院公报

## 6. 回答(Answer)
结合检索出来的文章片段对用户提问进行回答。

6.1 结合知识库回答

In [62]:
PROMPT_TEMPLATE = "基于以下已知信息，请简洁并专业地回答用户的问题。如果无法从中得到答案，请说 '根据已知信息无法回答该问题' 或 '没有提供足够的相关信息'。不允许在答案中添加编造成分。你可以参考以下文章:\n{DOCS}\n问题：{QUERY}\n回答："

chat_message = [
    {'role': 'user', 'content': PROMPT_TEMPLATE.format(DOCS=related_doc, QUERY=query)}
]
response = erniebot.ChatCompletion.create(model='ernie-bot-4', 
                                          messages=chat_message)
print(response.get_result())

国务院根据宪法和法律制定行政法规。具体流程包括以下几个步骤：

1. 立项：国务院有关部门认为需要制定行政法规的，应当向国务院报请立项。
2. 起草：行政法规由国务院有关部门或者国务院法制机构具体负责起草。重要行政管理的行政法规草案由国务院法制机构组织起草。在起草过程中，应当广泛听取有关机关、组织、人民代表大会代表和社会公众的意见。听取意见可以采取座谈会、论证会、听证会等多种形式。
3. 审查：行政法规草案完成后，起草单位应当将草案及其说明、各方面对草案主要问题的不同意见和其他有关资料送国务院法制机构进行审查。国务院法制机构应当向国务院提出审查报告和草案修改稿，审查报告应当对草案主要问题作出说明。
4. 决定和公布：行政法规的决定程序依照中华人民共和国国务院组织法的有关规定办理。行政法规由总理签署国务院令公布。行政法规签署公布后，及时在国务院公报和中国政府法制信息网以及在全国范围内发行的报纸上刊载。在国务院公报上刊登的行政法规文本为标准文本。

以上是国务院制定行政法规的基本流程，需要注意的是，行政法规的制定应当遵循宪法和法律的规定，同时也需要广泛听取各方意见，确保行政法规的科学性和民主性。


6.2 不结合知识库直接回答

In [111]:
chat_message = [
    {'role': 'user', 'content':query}
]
response = erniebot.ChatCompletion.create(model='ernie-bot-4', 
                                          messages=chat_message)
print(response.get_result())

国务院制定行政法规，应当根据宪法和法律，不得与宪法、法律相抵触。
国务院可以根据宪法和法律，制定行政法规。行政法规可以就下列事项作出规定：为执行法律的规定需要制定行政法规的事项；宪法第八十九条规定的国务院行政管理职权的事项。应当由全国人民代表大会及其常务委员会制定法律的事项，国务院根据全国人民代表大会及其常务委员会的授权决定先制定的行政法规，经过实践检验，制定法律的条件成熟时，国务院应当及时提请全国人民代表大会及其常务委员会制定法律。
